## Setup Environment

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
from ultralytics import YOLO , solutions
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load the YOLO Model

In [ ]:
# model = YOLO('/content/YOLO_Model.pt')
model = YOLO('/content/license_plate_detector.pt')

## Prepare the Video Capture

In [ ]:
# # Prepare the video capture
# video_path = '/content/drive/MyDrive/Dataset/sample.mp4'
# cap = cv2.VideoCapture(video_path)

In [ ]:
# Prepare the video capture
video_path = '/content/drive/MyDrive/Dataset/8321868-hd_1920_1080_30fps.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
print("Height:", height)
print("Width:", width)
print("FPS:", fps)

Height: 1080
Width: 1920
FPS: 30


In [ ]:
output_path = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))


In [ ]:
Licence_plate = set()

In [ ]:
import os

In [ ]:
counter = 0


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model.track(frame, persist=True)

    for result in results:
        for boxes in result.boxes:

            # Get Object details
            bbox = boxes.xyxy[0].cpu().numpy()          #Bounding Box
            class_id = int(boxes.cls[0].cpu().numpy())  #Class ID
            conf = boxes.conf[0].cpu().numpy()          #Confidence
            #Check if boxes.id is not None before accessing it
            if boxes.id is not None:
                id = int(boxes.id[0].cpu().numpy())         #Object ID
            else:
                #Handle the case where id is None
                id = -1

            x1, y1, x2, y2 = map(int, bbox)
            # Get the center of the Object
            # center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            cropped_object = frame[y1:y2, x1:x2]

            # Draw BBox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            label = f'ID: {id} Conf: {conf:.2f}'
            # Add labels for each car
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            # detect car and the id to the set
            if id not in Licence_plate:
                Licence_plate.add(id)
                save_path = os.path.join('/content/vid 3 images', f"{counter}.jpg")  # Save with object name and index
                counter += 1
                # Save the cropped image
                cv2.imwrite(save_path, cropped_object)

    out.write(frame)

    # Display each frame
    # cv2_imshow(frame)



requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 3 license_plates, 412.4ms
Speed: 33.2ms preprocess, 412.4ms inference, 30.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 246.8ms
Speed: 4.2ms preprocess, 246.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 229.8ms
Speed: 4.0ms preprocess, 229.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 228.0ms
Speed: 4.7ms preprocess, 228.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 252.7ms
Speed: 9.8ms preprocess, 252.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 64

KeyboardInterrupt: 

## Save and Submit Your Work

Submit both the notebook and the output video

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()
